In [46]:
# Load everything we need
from selenium import webdriver
from datetime import datetime, timedelta, date
import time
import email
import imaplib
import mailbox
import pandas as pd
import re

# import other web drivers that let you wait for elements to load
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [47]:
# this is the date of our reservation
newTime = '2020-03-06 10:00:00'

timestamp = time.mktime(time.strptime(newTime, '%Y-%m-%d %H:%M:%S'))
new_timestamp = ((datetime.fromtimestamp(timestamp) + timedelta(hours=2)).strftime('%-I:%M%p %A, %B %-d, %Y'))
hours = 6 # we need 1 account per 2 hours
times = int(hours/2)
room = 117 # the number of the study room to reserve
dt = new_timestamp
title = dt + " - " + str(room) + " - Available"
title = title.replace("PM","pm")
title = title.replace("AM","am")
newTitle = '[title="'+title+'"]'
print(newTitle)

d0 = date.today() 
d1 = date.fromtimestamp(timestamp)
delta = d1 - d0
days = int(delta.days)
# print(days)

# initialize list of e-mails
data = [['sensitive info', 'removed'], 
        ['sensitive info', 'removed'],  
        ['sensitive info', 'removed'], 
       ['sensitive info', 'removed'] ]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['email', 'pw'])

[title="12:00pm Friday, March 6, 2020 - 117 - Available"]


In [48]:
# initalize Safari driver
driver = webdriver.Safari()

for y in range(times):
    # open up library URL
    driver.get("https://southseattle.libcal.com/spaces?lid=5855&gid=10289")

    # initialize wait
    wait = WebDriverWait(driver, 2)

    # find and click on next day button
    if(days > 0):
        for x in range(days):
            nextDay = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[class="fc-next-button fc-button fc-button-primary"]')))
            #nextDay = driver.find_element_by_xpath('//*[@class="fc-next-button fc-button fc-button-primary"]')
            driver.execute_script("arguments[0].click();", nextDay)

    # wait and find and click on first time
    

    timestamp = time.mktime(time.strptime(newTime, '%Y-%m-%d %H:%M:%S'))
    new_timestamp = ((datetime.fromtimestamp(timestamp) + timedelta(hours=((y+1)*2)-2)).strftime('%-I:%M%p %A, %B %-d, %Y'))
    hours = 6 # we need 1 account per 2 hours
    times = int(hours/2)
    room = 117 # the number of the study room to reserve
    dt = new_timestamp
    title = dt + " - " + str(room) + " - Available"
    title = title.replace("PM","pm")
    title = title.replace("AM","am")
    newTitle = '[title="'+title+'"]'
    print(newTitle)
    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, newTitle)))
    driver.execute_script("arguments[0].click();", element)

    # wait and find and click on submit time
    waitForSubmitTime = wait.until(EC.element_to_be_clickable((By.ID, 'submit_times')))
    submitTime = driver.find_element_by_id('submit_times')
    driver.execute_script("arguments[0].click();", submitTime)

    # wait and find text fields
    waitForFields = wait.until(EC.element_to_be_clickable((By.ID, 'fname')))

    # email 
    email = df.iloc[(y-1)]['email']

    # enter information
    driver.find_element_by_id("fname").send_keys("Jon")
    driver.find_element_by_id("lname").send_keys("Caceres")
    driver.find_element_by_id("email").send_keys(email)
    driver.find_element_by_id("q5877").send_keys("Physics")
    driver.find_element_by_id("q5878").send_keys("4")

    checkBox = driver.find_element_by_xpath('//input[@name="q5881[]"]')
    checkBox2 = driver.find_element_by_xpath('//input[@name="q9595[]"]')
    driver.execute_script("arguments[0].click();", checkBox)
    driver.execute_script("arguments[0].click();", checkBox2)
    submit = driver.find_element_by_id("btn-form-submit")
    driver.execute_script("arguments[0].click();", submit)
    
    
    
    EMAIL_ACCOUNT = df.iloc[y-1]['email']
    PASSWORD = df.iloc[y-1]['pw']
    print(EMAIL_ACCOUNT)
    
    print("Reserved room... waiting 10 seconds to login...")
    print("Email to log into : " + EMAIL_ACCOUNT)
    for f in range(10):
        time.sleep(1)
        print(f)
    
    if((y-1)==1):
        mail = imaplib.IMAP4_SSL('imap.gmail.com')
    else:
        mail = imaplib.IMAP4_SSL('eas.outlook.com')
        
    mail.login(EMAIL_ACCOUNT, PASSWORD)
    
    mail.list()
    # Out: list of "folders" aka labels in gmail.
    mail.select("inbox") # connect to inbox.

    result, data_email = mail.search(None, '(FROM "alerts@mail.libcal.com")')

    ids = data_email[0] # data is a list.
    id_list = ids.split() # ids is a space separated string
    latest_email_id = id_list[-1] # get the latest

    result, data_email = mail.fetch(latest_email_id, "(RFC822)") # fetch the email body (RFC822) for the given ID

    raw_email = data_email[0][1] # here's the body, which is raw text of the whole email
    # including headers and alternate payloads
    
    raw_email = raw_email.decode("utf-8") 
    raw_email = str(raw_email.replace('\r', ''))

    data_email = raw_email
    
    WEB_URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com)\b/?(?!@)))"""
    links = re.findall(WEB_URL_REGEX, data_email)
    #print(links)
    url = "equipment/confirm/cs_"

    for idx, word in enumerate(links):
        if url in word:
            urlToOpen = word;
        else:
            continue

    #print(urlToOpen)


    urlToOpen = urlToOpen.partition("confirm/")[2]
    finalUrl = 'https://southseattle.libcal.com/equipment/confirm/' + urlToOpen
    finalUrl = finalUrl.replace('=','')

    print(finalUrl)
    
    driver.get(finalUrl)
    
    # waitToVerify = wait.until(EC.element_to_be_clickable((By.ID, 's-lc-verify-button')))
    verify = driver.find_element_by_id('s-lc-verify-button')
    driver.execute_script("arguments[0].click();", verify)

    
driver.close()

[title="10:00am Friday, March 6, 2020 - 117 - Available"]
ngoc.n.nguyen@seattlecolleges.edu
Reserved room... waiting 10 seconds to login...
Email to log into : ngoc.n.nguyen@seattlecolleges.edu
0
1
2
3
4
5
6
7
8
9
https://southseattle.libcal.com/equipment/confirm/cs_5vPXPA
[title="12:00pm Friday, March 6, 2020 - 117 - Available"]
jonathan.caceres@seattlecolleges.edu
Reserved room... waiting 10 seconds to login...
Email to log into : jonathan.caceres@seattlecolleges.edu
0
1
2
3
4
5
6
7
8
9
https://southseattle.libcal.com/equipment/confirm/cs_DOx9xlUX
